In [1]:
import os
import pandas as pd 
import glob
from collections import Counter

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
glob.glob('*.csv')

['cons_wb_go_edu_demos.csv',
 'income.csv',
 'cons_wb_go_edu_demos_inc_pov_hou_rem.csv',
 'edu_hh_ind_merge.csv',
 'pov_hou_rem_merged_rev4.csv',
 'wellbeing_goingson.csv',
 'goingson.csv',
 'demos_merged.csv',
 'cons_wb_go_edu_demos_inc.csv',
 'pov_hou_rem_merge_rev4.csv',
 'wellbeing.csv',
 'pov_hou_rem_merge_rev1.csv',
 'consumption_mapped_to_m_ids.csv']

In [4]:
trx = pd.read_csv('../diaries_transactions_all.csv', low_memory = False, dtype={'m_ids_owner': object})

In [18]:
cons = pd.read_csv('consumption_mapped_to_m_ids.csv', dtype={'m_ids_owner': object})
demos = pd.read_csv('demos_merged.csv',  dtype={'m_ids': object})
rem_hou_pov = pd.read_csv('pov_hou_rem_merged_rev4.csv',  dtype={'m_ids_owner': object}, index_col = 0)
wb_go = pd.read_csv('wellbeing_goingson.csv',  dtype={'hh_ids': object})
edu = pd.read_csv('edu_hh_ind_merge.csv', index_col = 0, dtype={'m_ids': object})
inc = pd.read_csv('income.csv', dtype={'m_ids_owner': object})

In [15]:
## we only want to keep IDs that are present in our main transaction datasets anyway
keep_ids_list = list(trx.m_ids_owner.unique())
keep_hh_list = list (trx.hh_ids.unique())

In [20]:
rem_hou_pov.head()

,Unnamed: 0.1,hh_ids,m_ids_owner,rem_i_unique_sources,rem_i_meanptrx,rem_i_sdptrx,rem_i_meanptrx_RG,rem_i_sdptrx_RG,rem_i_meanptrx_RR,rem_i_sdptrx_RR,rem_i_count_medianpti,rem_i_abs_medianpti,rem_i_net_medianpti,rem_i_count_medianpti_RG,rem_i_abs_medianpti_RG,rem_i_count_medianpti_RR,rem_i_abs_medianpti_RR,rem_i_count_medianpti_cash,rem_i_abs_medianpti_cash,rem_i_net_medianpti_cash,rem_i_count_medianpti_inkind,rem_i_abs_medianpti_inkind,rem_i_net_medianpti_inkind,rem_i_count_medianpti_mm,rem_i_abs_medianpti_mm,rem_i_net_medianpti_mm,rem_i_count_medianpti_other,rem_i_abs_medianpti_other,rem_i_net_medianpti_other,inc_hh_medianpti,con_pur_hh_medpti,con_prod_hh_medpti,dem_urban,dem_hh_size,dem_pae_oecd,dem_pae_kihbs,con_hh_medpti,inc_hh_pti_pae_oecd,con_hh_pti_pae_oecd,inc_hh_pti_pae_kihbs,con_hh_pti_pae_kihbs,inc_below_85_kes_oecd,con_below_85_kes_oecd,inc_below_85_kes_kihbs,con_below_85_kes_kihbs,inc_below_170_kes_oecd,con_below_170_kes_oecd,inc_below_170_kes_kihbs,con_below_170_kes_kihbs,inc_below_425_kes_oecd,con_below_425_kes_oecd,inc_below_425_kes_kihbs,con_below_425_kes_kihbs,monthly_left,hh_inf_wall_material,hh_inf_roof_material,hh_inf_floor_material,hh_inf_cookfuel_source,hh_inf_lighting_source,hh_inf_water_source,hh_inf_rooms_habitable,hh_inf_rooms_other,hh_inf_toilet_type,hh_inf_toilet_shared,hh_inf_housing_type,hh_inf_electricity_access,hh_inf_interiorcond,hh_asset_internet_access,hh_env_settingdesc,hh_env_dilapbuildings,hh_ownership_docu,hh_otherprop_any,hh_otherprop_N,hh_othershamba_N,add_prop_value
0,0,KELDK02,5.6134389503e+16,1.0,4606.4210,2317.3330,4606.421,2317.333,NaN,NaN,2.0,6983.0,-6983.0,2.0,6983.0,0.0,0.0,1.0,3135.0,-3135.0,0.0,0.0,0.0,1.0,2500.0,-2500.0,0.0,0.0,0.0,35472.5,10665.0,0.000,0.0,3.0,2.4,3.00,10665.000,484.59700,145.69670,387.67760,116.55740,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,24807.500,3,3.0,5.0,2.0,6,5.0,1.0,1.0,1.0,0.0,3.0,2.0,3.0,3.0,0.0,2.0,NaN,1.0,1.0,NaN,NaN
1,1,KELDK08,5.61344107097e+16,2.0,428.5000,139.3000,NaN,NaN,330.0000,NaN,1.0,428.5,-98.5,0.5,263.5,0.5,165.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,428.5,-98.5,0.0,0.0,0.0,3520.0,2890.0,0.000,0.0,3.0,2.2,2.24,2890.000,52.45901,43.07005,51.52225,42.30094,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,630.000,3,3.0,5.0,1.0,4,2.0,1.0,1.0,1.0,0.0,3.0,0.0,3.0,0.0,0.0,2.0,NaN,1.0,1.0,NaN,NaN
2,2,KELDK09,5.61343299446e+16,5.0,656.1765,1560.7240,5000.000,1732.051,NaN,NaN,2.0,750.0,30.0,0.0,0.0,2.0,300.0,0.0,0.0,0.0,2.0,300.0,30.0,0.0,0.0,0.0,0.0,0.0,0.0,8400.0,6415.0,0.000,0.0,2.0,1.5,1.65,6415.000,183.60660,140.21860,166.91510,127.47140,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1985.000,2,3.0,5.0,4.0,4,2.0,1.0,1.0,1.0,0.0,3.0,0.0,1.0,3.0,0.0,2.0,NaN,0.0,NaN,NaN,NaN
3,3,KELDK10,5.61343371572e+16,6.0,166.8349,618.0686,NaN,NaN,114.8535,503.7492,29.0,3427.0,1440.0,0.5,100.0,27.5,1540.0,1.0,600.0,-100.0,27.0,950.0,570.0,0.0,0.0,0.0,0.0,0.0,0.0,5105.0,6005.0,0.000,0.0,1.0,1.0,1.00,6005.000,167.37700,196.88520,167.37700,196.88520,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,-900.000,4,3.0,5.0,2.0,4,2.0,1.0,1.0,1.0,0.0,3.0,0.0,2.0,1.0,0.0,2.0,NaN,1.0,1.0,NaN,NaN
4,4,KELDK12,6.51344362288e+16,1.0,2000.0000,NaN,2000.000,NaN,NaN,NaN,1.0,2000.0,-2000.0,1.0,2000.0,0.0,0.0,1.0,2000.0,-2000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3700.0,2680.0,1217.857,0.0,4.0,2.7,2.89,3897.857,44.93018,47.33281,41.97629,44.22097,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-197.857,2,3.0,1.0,1.0,4,2.0,2.0,2.0,1.0,1.0,2.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,NaN,NaN,NaN


In [16]:
print(cons.shape)
print(demos.shape)
print(wb_go.shape)
print(rem_hou_pov.shape)
print(edu.shape)
print(inc.shape)
print(trx.shape)

(670, 38)
(669, 29)
(298, 15)
(460, 75)
(669, 12)
(643, 23)
(483949, 58)


In [10]:
cons = cons.loc[cons['m_ids_owner'].isin(keep_ids_list)]
edu = edu.loc[edu['m_ids'].isin(keep_ids_list)]
inc = inc.drop_duplicates()
rem_hou_pov = rem_hou_pov.loc[rem_hou_pov['m_ids_owner'].isin(keep_ids_list)]
wb_go = wb_go.loc[wb_go['hh_ids'].isin(keep_hh_list)] 

In [11]:
demos = demos.loc[demos['m_ids'].isin(keep_ids_list)]

In [12]:
print(cons.shape)
print(wb_go.shape)
print(edu.shape)
print(inc.shape)
print(demos.shape)
print(rem_hou_pov.shape)

(670, 38)
(298, 15)
(669, 12)
(643, 23)
(669, 29)
(0, 75)


In [64]:
inc.head()

,Unnamed: 0,hh_ids,m_ids_owner,hh_income,hh_income_only,inc_i_unique_sources,inc_i_minpti,inc_i_meanpti,inc_i_sdpti,inc_i_maxpti,...,inc_i_sdpti_CITMPC,inc_i_meanpti_NEINCG,inc_i_sdpti_NEINCG,inc_i_meanpti_REI,inc_i_sdpti_REI,inc_i_meanpti_RIOI,inc_i_sdpti_RIOI,inc_i_meanpti_SEI,inc_i_sdpti_SEI,inc_i_dom
0,0,KELDK01,56134390476100000,0.0,0.0,2.0,17.647060,103.11550,35.65044,137.23530,...,22.606590,0.0,0.0,67.45454,26.98390,0.0,0.0,0.0000,0.0000,REI
1,1,KELDK02,56134389503000000,0.0,0.0,1.0,-1529.412000,356.43330,1037.63000,2001.17700,...,0.000000,0.0,0.0,0.00000,0.00000,0.0,0.0,356.4333,1037.6300,SEI
2,2,KELDK08,56134410699600000,0.0,0.0,2.0,2.352941,30.01307,17.49300,51.88235,...,7.638482,0.0,0.0,21.83006,17.44106,0.0,0.0,0.0000,0.0000,REI
3,3,KELDK08,56134410709700000,0.0,0.0,2.0,3.764706,25.14804,14.33156,41.17647,...,4.155897,0.0,0.0,22.35294,17.06343,0.0,0.0,0.0000,0.0000,REI
4,4,KELDK09,56134329944600000,0.0,0.0,1.0,-2.352941,123.48660,138.51580,434.11760,...,0.000000,0.0,0.0,0.00000,0.00000,0.0,0.0,123.4866,138.5158,SEI


In [65]:
cons.head()

,m_ids_owner,hh_ids,con_pur_hh_meanptrx,con_pur_hh_sdptrx,con_pur_hh_minpti,con_pur_hh_meanpti,con_pur_hh_medpti,con_pur_hh_maxpti,con_pur_hh_meanpti_ALC,con_pur_hh_medpti_ALC,...,con_pur_hh_meanpti_MED,con_pur_hh_medpti_MED,con_pur_hh_meanpti_OTH,con_pur_hh_medpti_OTH,con_pur_hh_meanpti_PERS,con_pur_hh_medpti_PERS,con_pur_hh_meanpti_REL,con_pur_hh_medpti_REL,con_pur_hh_meanpti_TRNSP,con_pur_hh_medpti_TRNSP
0,58134383397900000,KELDL02,107.9147,286.7984,3293,14372.27,15640.0,23670,204.5455,0.0,...,27.27273,0,14.09091,0.0,274.72730,286.0,64.54546,20.0,487.7273,550.0
1,58134383535800000,KELDL02,107.9147,286.7984,3293,14372.27,15640.0,23670,204.5455,0.0,...,27.27273,0,14.09091,0.0,274.72730,286.0,64.54546,20.0,487.7273,550.0
2,HH,KVIHC16,115.6588,345.8901,1070,5825.00,5470.0,15380,0.0000,0.0,...,50.00000,0,27.27273,0.0,10.90909,0.0,15.45455,0.0,173.6364,60.0
3,59134423954300000,KVIHC16,115.6588,345.8901,1070,5825.00,5470.0,15380,0.0000,0.0,...,50.00000,0,27.27273,0.0,10.90909,0.0,15.45455,0.0,173.6364,60.0
4,65134441430300000,KELDK21,1061.1900,4257.9390,1150,26529.75,17300.0,75020,0.0000,0.0,...,16.66667,0,2018.50000,0.0,90.83334,0.0,816.66670,100.0,1198.3330,550.0


In [66]:
wb_go.head()

,hh_ids,happiness,economically,relationships,confidence,police,doctor,unsafe,utilities,asset_taken,miss_app,miss_inc,romantic,hungry,school_sent
0,KELDK01,4.045455,3.863636,4.863636,4.681818,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.071429
1,KELDK02,4.105263,3.947368,4.842105,4.736842,0.0,0.0,0.076923,0.0,0.0,0.0,0.153846,0.000000,0.000000,0.111111
2,KELDK08,4.038462,3.653846,4.730769,4.884615,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.058824,0.142857
3,KELDK09,4.052632,3.947368,4.947368,4.894737,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.071429
4,KELDK10,4.111111,4.000000,4.777778,4.944444,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.058824,0.000000,0.000000


In [67]:
edu.head()

,m_ids,hh_ids,edu_i_enrolled,edu_i_attain,edu_i_attain_yrs,edu_hh_child613inschool,edu_hh_child1417inschool,edu_hh_girls613inschool,edu_hh_girls1417inschool,edu_hh_boys613inschool,edu_hh_boys1417inschool,edu_hh_headattain_yrs
0,56134390476100000,KELDK01,0.0,Primary (some or complete),8.0,1.0,NaN,1.0,NaN,NaN,NaN,8.0
1,56134390654900000,KELDK01,0.0,Secondary (some or complete),10.0,1.0,NaN,1.0,NaN,NaN,NaN,8.0
2,56134389503000000,KELDK02,0.0,Primary (some or complete),8.0,NaN,1.0,NaN,1.0,NaN,NaN,8.0
3,56134410699600000,KELDK08,0.0,Primary (some or complete),5.0,0.0,NaN,NaN,NaN,0.0,NaN,5.0
4,56134410709700000,KELDK08,0.0,Primary (some or complete),8.0,0.0,NaN,NaN,NaN,0.0,NaN,5.0


In [68]:
demos.head()

,hh_ids,m_ids,dem_i_male,dem_i_age_yrs,dem_i_age_5yrgrp,dem_i_age_workage,dem_i_relhead,dem_i_hhead,dem_i_marstat,dem_i_tribe,...,dem_hh_ages15plus,dem_hh_ages18plus,dem_hh_ages65plus,dem_hh_ages20_30,dem_hh_workage,dem_hh_dependents,dem_hh_agehead,dem_hh_malehead,dem_pae_oecd,dem_pae_kihbs
0,KELDK01,56134390476100000,Male,30,[30 - 35),1,Household head,1,Married/living together,Kalenjin,...,2,2,0,2,0.500000,0.500000,30,1,2.7,2.89
1,KELDK01,56134390654900000,Female,24,[20 - 25),1,Husband or wife,0,Married/living together,Kalenjin,...,2,2,0,2,0.500000,0.500000,30,1,2.7,2.89
2,KELDK02,56134389503000000,Male,42,[40 - 45),1,Household head,1,Married/living together,Luhya,...,3,2,0,0,1.000000,0.000000,42,1,2.4,3.00
3,KELDK08,56134410709700000,Female,20,[20 - 25),1,Husband or wife,0,Married/living together,Kalenjin,...,2,2,0,2,0.666667,0.333333,26,1,2.2,2.24
4,KELDK08,56134410699600000,Male,26,[25 - 30),1,Household head,1,Married/living together,Luhya,...,2,2,0,2,0.666667,0.333333,26,1,2.2,2.24


## Merging: 

In [69]:
cons_wb_go = cons.merge(wb_go, on = 'hh_ids', how = 'left')

In [70]:
cons_wb_go.head()

,m_ids_owner,hh_ids,con_pur_hh_meanptrx,con_pur_hh_sdptrx,con_pur_hh_minpti,con_pur_hh_meanpti,con_pur_hh_medpti,con_pur_hh_maxpti,con_pur_hh_meanpti_ALC,con_pur_hh_medpti_ALC,...,police,doctor,unsafe,utilities,asset_taken,miss_app,miss_inc,romantic,hungry,school_sent
0,58134383397900000,KELDL02,107.9147,286.7984,3293,14372.27,15640.0,23670,204.5455,0.0,...,0.111111,0.000000,0.0,0.0,0.0,0.000000,0.055556,0.0,0.000000,0.000000
1,58134383535800000,KELDL02,107.9147,286.7984,3293,14372.27,15640.0,23670,204.5455,0.0,...,0.111111,0.000000,0.0,0.0,0.0,0.000000,0.055556,0.0,0.000000,0.000000
2,HH,KVIHC16,115.6588,345.8901,1070,5825.00,5470.0,15380,0.0000,0.0,...,0.000000,0.157895,0.0,0.0,0.0,0.052632,0.052632,0.0,0.105263,0.272727
3,59134423954300000,KVIHC16,115.6588,345.8901,1070,5825.00,5470.0,15380,0.0000,0.0,...,0.000000,0.157895,0.0,0.0,0.0,0.052632,0.052632,0.0,0.105263,0.272727
4,65134441430300000,KELDK21,1061.1900,4257.9390,1150,26529.75,17300.0,75020,0.0000,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.166667,0.0,0.000000,0.100000


In [71]:
cons_wb_go.shape

(670, 52)

In [72]:
cons_wb_go_edu = cons_wb_go.merge(edu, left_on  = ['m_ids_owner', 'hh_ids'], right_on=['m_ids', 'hh_ids'], how = 'outer')


In [73]:
cons_wb_go_edu.shape

(670, 63)

In [74]:
cons_wb_go_edu.sort_values('hh_ids').head()

,m_ids_owner,hh_ids,con_pur_hh_meanptrx,con_pur_hh_sdptrx,con_pur_hh_minpti,con_pur_hh_meanpti,con_pur_hh_medpti,con_pur_hh_maxpti,con_pur_hh_meanpti_ALC,con_pur_hh_medpti_ALC,...,edu_i_enrolled,edu_i_attain,edu_i_attain_yrs,edu_hh_child613inschool,edu_hh_child1417inschool,edu_hh_girls613inschool,edu_hh_girls1417inschool,edu_hh_boys613inschool,edu_hh_boys1417inschool,edu_hh_headattain_yrs
209,56134390654900000,KELDK01,83.46289,228.754,1470,7122.167,8805.0,10055,0.0,0.0,...,0.0,Secondary (some or complete),10.0,1.0,NaN,1.0,NaN,NaN,NaN,8.0
208,56134390476100000,KELDK01,83.46289,228.754,1470,7122.167,8805.0,10055,0.0,0.0,...,0.0,Primary (some or complete),8.0,1.0,NaN,1.0,NaN,NaN,NaN,8.0
10,56134389503000000,KELDK02,321.83330,1245.083,5815,12873.330,10665.0,31300,0.0,0.0,...,0.0,Primary (some or complete),8.0,NaN,1.0,NaN,1.0,NaN,NaN,8.0
164,56134410699600000,KELDK08,47.59043,142.991,800,3137.308,2890.0,4590,0.0,0.0,...,0.0,Primary (some or complete),5.0,0.0,NaN,NaN,NaN,0.0,NaN,5.0
163,56134410709700000,KELDK08,47.59043,142.991,800,3137.308,2890.0,4590,0.0,0.0,...,0.0,Primary (some or complete),8.0,0.0,NaN,NaN,NaN,0.0,NaN,5.0


In [75]:
cons_wb_go_edu = cons_wb_go_edu.loc[cons_wb_go_edu['m_ids_owner'].isin(keep_ids_list)]

In [76]:
cons_wb_go_edu.shape

(670, 63)

In [77]:
len(cons_wb_go_edu.m_ids_owner.unique())

670

In [78]:
inc = inc.loc[inc['m_ids_owner'].isin(keep_ids_list)]

In [79]:
inc.shape

(582, 23)

In [80]:
len(inc.m_ids_owner.unique())

582

In [81]:
cons_wb_go_edu_demos = cons_wb_go_edu.merge(demos, left_on  = ['m_ids_owner', 'hh_ids'], right_on =['m_ids', 'hh_ids'], how = 'outer')

In [82]:
# cons_wb_go_edu_demos = cons_wb_go_edu.merge(demos, on  = ['m_ids_owner', 'hh_ids'], how = 'outer')

In [83]:
cons_wb_go_edu_demos.shape

(670, 91)

In [84]:
cons_wb_go_edu_demos.to_csv('cons_wb_go_edu_demos.csv', index = False)

## Adding income & housing/poverty 

In [85]:
cons_wb_go_edu_demos_inc = cons_wb_go_edu_demos.merge(inc, on = ['m_ids_owner', 'hh_ids'], how = 'left')

In [86]:
cons_wb_go_edu_demos_inc.shape

(670, 112)

In [87]:
cons_wb_go_edu_demos_inc.head()

,m_ids_owner,hh_ids,con_pur_hh_meanptrx,con_pur_hh_sdptrx,con_pur_hh_minpti,con_pur_hh_meanpti,con_pur_hh_medpti,con_pur_hh_maxpti,con_pur_hh_meanpti_ALC,con_pur_hh_medpti_ALC,...,inc_i_sdpti_CITMPC,inc_i_meanpti_NEINCG,inc_i_sdpti_NEINCG,inc_i_meanpti_REI,inc_i_sdpti_REI,inc_i_meanpti_RIOI,inc_i_sdpti_RIOI,inc_i_meanpti_SEI,inc_i_sdpti_SEI,inc_i_dom
0,58134383397900000,KELDL02,107.9147,286.7984,3293,14372.27,15640.0,23670,204.5455,0.0,...,49.97194,0.0,0.0,0.00000,0.00000,12.299470,21.119950,0.0,0.0,CITMPC
1,58134383535800000,KELDL02,107.9147,286.7984,3293,14372.27,15640.0,23670,204.5455,0.0,...,0.00000,0.0,0.0,58.82353,58.82353,0.392157,0.679236,0.0,0.0,REI
2,HH,KVIHC16,115.6588,345.8901,1070,5825.00,5470.0,15380,0.0000,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,59134423954300000,KVIHC16,115.6588,345.8901,1070,5825.00,5470.0,15380,0.0000,0.0,...,NaN,0.0,NaN,0.00000,NaN,0.000000,NaN,0.0,NaN,AGRI
4,65134441430300000,KELDK21,1061.1900,4257.9390,1150,26529.75,17300.0,75020,0.0000,0.0,...,0.00000,0.0,0.0,0.00000,0.00000,0.000000,0.000000,0.0,0.0,AGRI


In [88]:
cons_wb_go_edu_demos_inc.to_csv('cons_wb_go_edu_demos_inc.csv', index = False)

## Here we will add pov hou and rem when Dinara is through: 

In [89]:
len([x for x in rem_hou_pov.m_ids_owner if str(x) in list(cons_wb_go_edu_demos_inc.m_ids_owner)])

450

In [90]:
len(rem_hou_pov.m_ids_owner.unique())

450

In [97]:
rem_hou_pov.head()

,hh_ids,m_ids_owner,rem_i_unique_sources,rem_i_meanptrx,rem_i_sdptrx,rem_i_meanptrx_RG,rem_i_sdptrx_RG,rem_i_meanptrx_RR,rem_i_sdptrx_RR,rem_i_count_medianpti,rem_i_abs_medianpti,rem_i_net_medianpti,rem_i_count_medianpti_RG,rem_i_abs_medianpti_RG,rem_i_count_medianpti_RR,rem_i_abs_medianpti_RR,rem_i_count_medianpti_cash,rem_i_abs_medianpti_cash,rem_i_net_medianpti_cash,rem_i_count_medianpti_inkind,rem_i_abs_medianpti_inkind,rem_i_net_medianpti_inkind,rem_i_count_medianpti_mm,rem_i_abs_medianpti_mm,rem_i_net_medianpti_mm,rem_i_count_medianpti_other,rem_i_abs_medianpti_other,rem_i_net_medianpti_other,inc_hh_medianpti,dem_hh_size,con_hh_medpti,inc_hh_pti_pae_oecd,con_hh_pti_pae_oecd,inc_hh_pti_pae_kihbs,con_hh_pti_pae_kihbs,inc_below_85_kes_oecd,con_below_85_kes_oecd,inc_below_85_kes_kihbs,con_below_85_kes_kihbs,inc_below_170_kes_oecd,con_below_170_kes_oecd,inc_below_170_kes_kihbs,con_below_170_kes_kihbs,inc_below_425_kes_oecd,con_below_425_kes_oecd,inc_below_425_kes_kihbs,con_below_425_kes_kihbs,monthly_left,hh_inf_wall_material,hh_inf_roof_material,hh_inf_floor_material,hh_inf_cookfuel_source,hh_inf_lighting_source,hh_inf_water_source,hh_inf_rooms_habitable,hh_inf_rooms_other,hh_inf_toilet_type,hh_inf_toilet_shared,hh_inf_housing_type,hh_inf_electricity_access,hh_inf_interiorcond,hh_asset_internet_access,hh_otherprop_any,hh_otherprop_N,hh_otherprop1_value,hh_othershamba_N
0,KELDK02,56134389503000000,1,4606.4210,2317.3330,4606.421,2317.333,NaN,NaN,2.0,6983.0,-6983.0,2.0,6983.0,0.0,0.0,1.0,3135.0,-3135.0,0.0,0.0,0.0,1.0,2500.0,-2500.0,0.0,0,0,35472.5,3.0,10665.000,484.59700,145.69670,387.67760,116.55740,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,24807.500,3,3.0,5.0,2.0,6.0,5.0,1.0,1.0,1.0,0.0,3.0,2.0,3.0,3.0,1.0,1.0,100000.0,NaN
1,KELDK08,56134410709700000,2,428.5000,139.3000,NaN,NaN,330.0000,NaN,1.0,428.5,-98.5,0.5,263.5,0.5,165.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,428.5,-98.5,0.0,0,0,3520.0,3.0,2890.000,52.45901,43.07005,51.52225,42.30094,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,630.000,3,3.0,5.0,1.0,4.0,2.0,1.0,1.0,1.0,0.0,3.0,0.0,3.0,0.0,1.0,1.0,40000.0,NaN
2,KELDK09,56134329944600000,5,656.1765,1560.7240,5000.000,1732.051,NaN,NaN,2.0,750.0,30.0,0.0,0.0,2.0,300.0,0.0,0.0,0.0,2.0,300.0,30.0,0.0,0.0,0.0,0.0,0,0,8400.0,2.0,6415.000,183.60660,140.21860,166.91510,127.47140,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1985.000,2,3.0,5.0,4.0,4.0,2.0,1.0,1.0,1.0,0.0,3.0,0.0,1.0,3.0,0.0,NaN,NaN,NaN
3,KELDK10,56134337157200000,6,166.8349,618.0686,NaN,NaN,114.8535,503.7492,29.0,3427.0,1440.0,0.5,100.0,27.5,1540.0,1.0,600.0,-100.0,27.0,950.0,570.0,0.0,0.0,0.0,0.0,0,0,5105.0,1.0,6005.000,167.37700,196.88520,167.37700,196.88520,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,-900.000,4,3.0,5.0,2.0,4.0,2.0,1.0,1.0,1.0,0.0,3.0,0.0,2.0,1.0,1.0,1.0,25000.0,NaN
4,KELDK12,65134436228800000,1,2000.0000,NaN,2000.000,NaN,NaN,NaN,1.0,2000.0,-2000.0,1.0,2000.0,0.0,0.0,1.0,2000.0,-2000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,3700.0,4.0,3897.857,44.93018,47.33281,41.97629,44.22097,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-197.857,2,3.0,1.0,1.0,4.0,2.0,2.0,2.0,1.0,1.0,2.0,0.0,1.0,0.0,0.0,NaN,NaN,NaN


In [98]:
rem_hou_pov.shape

(450, 66)

In [99]:
rem_hou_pov.m_ids_owner.value_counts().head(10)

62136490111200000     1
61134395033200000     1
57134338167400000     1
64134408292500000     1
57134320879700000     1
65134364671700000     1
57137024579500000     1
108136868079500000    1
62134426712200000     1
57134320861700000     1
Name: m_ids_owner, dtype: int64

In [104]:
cons_wb_go_edu_demos_inc_pov_hou_rem = cons_wb_go_edu_demos_inc.merge(rem_hou_pov, on = ["m_ids_owner", 'hh_ids'],  how = 'left')

In [105]:
cons_wb_go_edu_demos_inc_pov_hou_rem.shape

(670, 176)

In [106]:
cons_wb_go_edu_demos_inc_pov_hou_rem.head()

,m_ids_owner,hh_ids,con_pur_hh_meanptrx,con_pur_hh_sdptrx,con_pur_hh_minpti,con_pur_hh_meanpti,con_pur_hh_medpti,con_pur_hh_maxpti,con_pur_hh_meanpti_ALC,con_pur_hh_medpti_ALC,con_pur_hh_meanpti_CLN,con_pur_hh_medpti_CLN,con_pur_hh_meanpti_CLTH,con_pur_hh_medpti_CLTH,con_pur_hh_meanpti_COMM,con_pur_hh_medpti_COMM,con_pur_hh_meanpti_EDU,con_pur_hh_medpti_EDU,con_pur_hh_meanpti_ENRGY,con_pur_hh_medpti_ENRGY,con_pur_hh_meanpti_ENT,con_pur_hh_medpti_ENT,con_pur_hh_meanpti_FOOD,con_pur_hh_medpti_FOOD,con_pur_hh_meanpti_H20,con_pur_hh_medpti_H20,con_pur_hh_meanpti_HSNG,con_pur_hh_medpti_HSNG,con_pur_hh_meanpti_MED,con_pur_hh_medpti_MED,con_pur_hh_meanpti_OTH,con_pur_hh_medpti_OTH,con_pur_hh_meanpti_PERS,con_pur_hh_medpti_PERS,con_pur_hh_meanpti_REL,con_pur_hh_medpti_REL,con_pur_hh_meanpti_TRNSP,con_pur_hh_medpti_TRNSP,happiness,economically,relationships,confidence,police,doctor,unsafe,utilities,asset_taken,miss_app,miss_inc,romantic,hungry,school_sent,m_ids_x,edu_i_enrolled,edu_i_attain,edu_i_attain_yrs,edu_hh_child613inschool,edu_hh_child1417inschool,edu_hh_girls613inschool,edu_hh_girls1417inschool,edu_hh_boys613inschool,edu_hh_boys1417inschool,edu_hh_headattain_yrs,m_ids_y,dem_i_male,dem_i_age_yrs,dem_i_age_5yrgrp,dem_i_age_workage,dem_i_relhead,dem_i_hhead,dem_i_marstat,dem_i_tribe,dem_i_children_tot,dem_i_children_inhh,dem_urban,dem_hh_size_x,dem_hh_meanage,dem_hh_ages0_4,dem_hh_ages5_14,dem_hh_ages15_17,dem_hh_ages0_17,dem_hh_ages15plus,dem_hh_ages18plus,dem_hh_ages65plus,dem_hh_ages20_30,dem_hh_workage,dem_hh_dependents,dem_hh_agehead,dem_hh_malehead,dem_pae_oecd,dem_pae_kihbs,Unnamed: 0,hh_income,hh_income_only,inc_i_unique_sources,inc_i_minpti,inc_i_meanpti,inc_i_sdpti,inc_i_maxpti,inc_i_meanpti_AGRI,inc_i_sdpti_AGRI,inc_i_meanpti_CITMPC,inc_i_sdpti_CITMPC,inc_i_meanpti_NEINCG,inc_i_sdpti_NEINCG,inc_i_meanpti_REI,inc_i_sdpti_REI,inc_i_meanpti_RIOI,inc_i_sdpti_RIOI,inc_i_meanpti_SEI,inc_i_sdpti_SEI,inc_i_dom,rem_i_unique_sources,rem_i_meanptrx,rem_i_sdptrx,rem_i_meanptrx_RG,rem_i_sdptrx_RG,rem_i_meanptrx_RR,rem_i_sdptrx_RR,rem_i_count_medianpti,rem_i_abs_medianpti,rem_i_net_medianpti,rem_i_count_medianpti_RG,rem_i_abs_medianpti_RG,rem_i_count_medianpti_RR,rem_i_abs_medianpti_RR,rem_i_count_medianpti_cash,rem_i_abs_medianpti_cash,rem_i_net_medianpti_cash,rem_i_count_medianpti_inkind,rem_i_abs_medianpti_inkind,rem_i_net_medianpti_inkind,rem_i_count_medianpti_mm,rem_i_abs_medianpti_mm,rem_i_net_medianpti_mm,rem_i_count_medianpti_other,rem_i_abs_medianpti_other,rem_i_net_medianpti_other,inc_hh_medianpti,dem_hh_size_y,con_hh_medpti,inc_hh_pti_pae_oecd,con_hh_pti_pae_oecd,inc_hh_pti_pae_kihbs,con_hh_pti_pae_kihbs,inc_below_85_kes_oecd,con_below_85_kes_oecd,inc_below_85_kes_kihbs,con_below_85_kes_kihbs,inc_below_170_kes_oecd,con_below_170_kes_oecd,inc_below_170_kes_kihbs,con_below_170_kes_kihbs,inc_below_425_kes_oecd,con_below_425_kes_oecd,inc_below_425_kes_kihbs,con_below_425_kes_kihbs,monthly_left,hh_inf_wall_material,hh_inf_roof_material,hh_inf_floor_material,hh_inf_cookfuel_source,hh_inf_lighting_source,hh_inf_water_source,hh_inf_rooms_habitable,hh_inf_rooms_other,hh_inf_toilet_type,hh_inf_toilet_shared,hh_inf_housing_type,hh_inf_electricity_access,hh_inf_interiorcond,hh_asset_internet_access,hh_otherprop_any,hh_otherprop_N,hh_otherprop1_value,hh_othershamba_N
0,58134383397900000,KELDL02,107.9147,286.7984,3293,14372.27,15640.0,23670,204.5455,0.0,365.5454,200.0,292.7273,0,1071.81800,1050.0,873.6364,0.0,1162.2730,1170.0,0.0,0,6401.545,6590.0,40.90909,0.0,3090.909,3000.0,27.27273,0,14.09091,0.0,274.72730,286.0,64.54546,20.0,487.7273,550.0,3.863636,3.272727,4.227273,4.00,0.111111,0.000000,0.0,0.0,0.0,0.000000,0.055556,0.0,0.000000,0.000000,58134383397900000,0.0,Secondary (some or complete),12.0,NaN,NaN,NaN,NaN,NaN,NaN,12.0,58134383397900000,Male,27.0,[25 - 30),1.0,Household head,1.0,Married/living together,Luo,2.0,0.0,1.0,4.0,14.00,2.0,0.0,0.0,2.0,2.0,2.0,0.0,2.0,0.50,0.50,27.0,1.0,2.7,2.48,55.0,0.0,0.0,2.0,51.764710,158.128300,63.

In [108]:
[x for x in list(cons_wb_go_edu_demos_inc_pov_hou_rem.columns) if 'm_ids' in x]

['m_ids_owner', 'm_ids_x', 'm_ids_y']

In [114]:
cons_wb_go_edu_demos_inc_pov_hou_rem = cons_wb_go_edu_demos_inc_pov_hou_rem.drop(['m_ids_x', 'm_ids_y'], axis = 1)

In [115]:
cons_wb_go_edu_demos_inc_pov_hou_rem.shape

(670, 174)

In [116]:
cons_wb_go_edu_demos_inc_pov_hou_rem.to_csv('cons_wb_go_edu_demos_inc_pov_hou_rem.csv', index = False)